# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [16]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [ ]:
# Use this cell for any import statements you add



---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [18]:
artists = {'robyn':"https://www.azlyrics.com/r/robyn.html",
           'cher':"https://www.azlyrics.com/c/cher.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: 
```
User-agent: *
Disallow: /lyricsdb/
Disallow: /song/
Allow: /

User-agent: 008
Disallow: /
```

The `robots.txt` file for AZLyrics allows general crawling of the site, except for two key areas: `/lyricsdb/` and `/song/`, which are disallowed. This means scraping individual lyrics and song-related pages might be restricted. However, the general artist page, which lists songs, seems to be allowed.


In [40]:
import os
import requests
import random
import time
from bs4 import BeautifulSoup
from collections import defaultdict

# Step 1: Artist and page dictionary
artists = {'robyn': "https://www.azlyrics.com/r/robyn.html", 'cher': "https://www.azlyrics.com/c/cher.html"}
lyrics_pages = defaultdict(list)

# Step 2: Scraping artist page for song links
for artist, artist_page in artists.items():
    r = requests.get(artist_page)
    time.sleep(5 + 10 * random.random())
    soup = BeautifulSoup(r.text, 'html.parser')

    for link in soup.find_all('a'):
        href = link.get('href')
        if href and 'lyrics' in href:
            lyrics_pages[artist].append(href)

# Step 3: Ensure 20 songs per artist
for artist, lp in lyrics_pages.items():
    assert len(set(lp)) > 20

# Step 4: Extract and save lyrics
def generate_filename_from_link(link):
    name = link.replace("https://", "").replace("http://", "").replace(".html", "")
    name = name.replace("://", "").replace(".", "_").replace("/", "_") + ".txt"
    return name

if not os.path.exists("lyrics"):
    os.mkdir("lyrics")

for artist, pages in lyrics_pages.items():
    os.makedirs(f"lyrics/{artist}", exist_ok=True)
    for page in pages[:20]:
        if page.startswith("//"):
            page = "https:" + page  # Ensure proper scheme
        r = requests.get(page)
        time.sleep(5 + 10 * random.random())
        soup = BeautifulSoup(r.text, 'html.parser')
        title = soup.find('h1').text if soup.find('h1') else "No Title"
        # lyrics_div = soup.find('div', attrs={'class': None, 'id': None})
        lyrics_div = soup.find('div', class_=None)  # Adjust this as needed
        if lyrics_div:
            lyrics = lyrics_div.get_text(separator="\n").strip()
            filename = generate_filename_from_link(page)
            with open(f"lyrics/{artist}/{filename}", 'w', encoding='utf-8') as f:
                f.write(title + '\n\n' + lyrics)


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [52]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.73 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [53]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [54]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For robyn we have 20 files.
For robyn we have roughly 41 words, 5 are unique.
For cher we have 20 files.
For cher we have roughly 41 words, 5 are unique.
